In [1]:
import spotipy

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
from spotipy.oauth2 import SpotifyOAuth

In [4]:
from textblob import TextBlob

In [5]:
import random

In [6]:
import pandas as pd

In [7]:
from IPython.display import display, HTML

In [8]:
!pip install transformers torch

In [9]:
from transformers import pipeline

In [10]:
!pip install sentence_splitter

In [11]:
from sentence_splitter import SentenceSplitter

In [12]:
!pip install numpy scikit-learn

In [13]:
import numpy as np

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
!pip install gliclass

In [17]:
from gliclass import GLiClassModel, ZeroShotClassificationPipeline

In [18]:
from transformers import AutoTokenizer

In [19]:
SPOTIPY_CLIENT_ID = 'f17bf600f0bb478d9a2d4a92abe94cba'

In [20]:
SPOTIPY_CLIENT_SECRET = '15202bade1954c7fb791b19d60824a11'

In [21]:
auth_manager = SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri="http://127.0.0.1:8888/callback", 
    scope="user-read-private",
    cache_path=".cache"  
)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [22]:
emotion_pipeline = pipeline(
    "text-classification",
    model="boltuix/bert-emotion",
    return_all_scores=False,
    device=0 
)

Device set to use cpu
C:\Anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [23]:
zero_shot = pipeline(
    "zero-shot-classification",
    model="valhalla/distilbart-mnli-12-3",
    device=0  
)

Device set to use cpu


In [24]:
def classify_emotion_basic(text):
    result = emotion_pipeline(text)[0]
    return result['label'].lower(), result['score']

In [25]:
zero_shot_labels = [
    "anger", "disgust", "fear", "sadness", "joy", "surprise", "neutral",
    "love", "embarrassment", "confusion", "curiosity", "excitement",
    "gratitude", "grief", "hope", "pride", "relief", "romance", "anxiety",
    "loneliness", "disappointment", "shame", "guilt", "trust"
]

In [26]:
def classify_emotion_refined(text):
    output = zero_shot([text], zero_shot_labels)[0]
    scored = list(zip(output["labels"], output["scores"]))
    top3 = sorted(scored, key=lambda x: x[1], reverse=True)[:3]
    return [(label.lower(), score) for label, score in top3]

In [27]:
def map_emotion_to_genre(emotion_list):
    for emotion, score in emotion_list:
        if emotion in emotion_map:
            return emotion_map[emotion]
    return ("confused", "experimental") 

In [28]:
test_inputs = [
    "I just got dumped and everything tastes like dust",
    "My friend surprised me with coffee this morning!",
    "I feel a bit hopeful about the future",
    "I’m embarrassed I tripped on my own shoelace"
]
for text in test_inputs:
    print(f"\nInput: {text}")
    emotions = classify_emotion_refined(text)
    for label, score in emotions:
        print(f"  {label}: {score:.2%}")


Input: I just got dumped and everything tastes like dust
  disappointment: 39.27%
  disgust: 13.84%
  embarrassment: 9.62%

Input: My friend surprised me with coffee this morning!
  surprise: 60.07%
  joy: 13.34%
  excitement: 8.76%

Input: I feel a bit hopeful about the future
  excitement: 54.42%
  hope: 14.73%
  joy: 7.64%

Input: I’m embarrassed I tripped on my own shoelace
  embarrassment: 52.88%
  shame: 36.74%
  disgust: 1.92%


In [29]:
emotion_map = {
    "anger":        ("furious",    "punk-rock"),     
    "disgust":      ("nauseated",  "goth"),          
    "fear":         ("anxious",    "ambient"),       
    "sadness":      ("heartbroken","blues"),
    "joy":          ("euphoric",   "electro-house"), 
    "surprise":     ("curious",    "experimental"),  
    "neutral":      ("balanced",   "chill"),
    "love":         ("romantic",   "indie-pop"),
    "embarrassment":("mortified",  "disco"),         
    "confusion":    ("lost",       "electronic"),
    "curiosity":    ("exploratory","world-music"),
    "excitement":   ("energetic",  "edm"),
    "gratitude":    ("uplifted",   "acoustic"),
    "grief":        ("devastated", "ambient"),       
    "hope":         ("optimistic", "pop"),
    "pride":        ("confident",  "hip-hop"),
    "relief":       ("liberated",  "soul"),
    "romance":      ("loving",     "singer-songwriter"),
    "anxiety":      ("tense",      "downtempo"),     
    "loneliness":   ("isolated",   "acoustic"),
    "disappointment":("pensive",   "indie"),
    "shame":        ("withdrawn",  "classical"),
    "guilt":        ("heavy-hearted","jazz"),
    "trust":        ("secure",     "r-n-b")
}

In [30]:
genre_fallbacks = {
    "punk-rock": "rock",
    "goth": "alternative",
    "ambient": "chill",
    "electro-house": "electronic",
    "experimental": "electronic",
    "indie-pop": "indie-pop",
    "disco": "dance",
    "edm": "electronic",
    "acoustic": "singer-songwriter",
    "indie": "alternative",
    "classical": "classical",
    "jazz": "jazz",
    "hip-hop": "hip-hop",
    "pop": "pop",
    "electronic": "electronic",
    "world-music": "world-music",
    "downtempo": "electronic",
    "r-n-b": "r-n-b",
    "folk": "folk",
    "soul": "soul"
}

In [31]:
VALID_SEEDS = {
    "acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "black-metal", "bluegrass",
    "blues", "bossanova", "brazil", "breakbeat", "british", "cantopop", "chicago-house", "children",
    "chill", "classical", "club", "comedy", "country", "dance", "dancehall", "death-metal", "deep-house",
    "detroit-techno", "disco", "disney", "drum-and-bass", "dub", "dubstep", "edm", "electro", "electronic",
    "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel", "goth", "grindcore", "groove",
    "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal", "hip-hop", "holidays",
    "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance",
    "j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal",
    "metal-misc", "metalcore", "minimal-techno", "movies", "mpb", "new-age", "new-release", "opera",
    "pagode", "party", "philippines-opm", "piano", "pop", "pop-film", "post-dubstep", "power-pop",
    "progressive-house", "psych-rock", "punk", "punk-rock", "r-n-b", "rainy-day", "reggae", "reggaeton",
    "road-trip", "rock", "rock-n-roll", "rockabilly", "romance", "sad", "salsa", "samba", "sertanejo",
    "show-tunes", "singer-songwriter", "ska", "sleep", "songwriter", "soul", "soundtracks", "spanish",
    "study", "summer", "swedish", "synth-pop", "tango", "techno", "trance", "trip-hop", "turkish",
    "work-out", "world-music"
}

In [32]:
def validate_emotion_pipeline():
    for emotion, (_, genre) in emotion_map.items():
        fallback = genre_fallbacks.get(genre, genre)
        if fallback not in VALID_SEEDS:
            print(f"Genre '{genre}' (fallback: '{fallback}') is NOT valid.")
        else:
            print(f"{emotion}: '{genre}' ➜ '{fallback}'")
validate_emotion_pipeline()

anger: 'punk-rock' ➜ 'rock'
disgust: 'goth' ➜ 'alternative'
fear: 'ambient' ➜ 'chill'
sadness: 'blues' ➜ 'blues'
joy: 'electro-house' ➜ 'electronic'
surprise: 'experimental' ➜ 'electronic'
neutral: 'chill' ➜ 'chill'
love: 'indie-pop' ➜ 'indie-pop'
embarrassment: 'disco' ➜ 'dance'
confusion: 'electronic' ➜ 'electronic'
curiosity: 'world-music' ➜ 'world-music'
excitement: 'edm' ➜ 'electronic'
gratitude: 'acoustic' ➜ 'singer-songwriter'
grief: 'ambient' ➜ 'chill'
hope: 'pop' ➜ 'pop'
pride: 'hip-hop' ➜ 'hip-hop'
relief: 'soul' ➜ 'soul'
romance: 'singer-songwriter' ➜ 'singer-songwriter'
anxiety: 'downtempo' ➜ 'electronic'
loneliness: 'acoustic' ➜ 'singer-songwriter'
disappointment: 'indie' ➜ 'alternative'
shame: 'classical' ➜ 'classical'
guilt: 'jazz' ➜ 'jazz'
trust: 'r-n-b' ➜ 'r-n-b'


In [33]:
def format_tracks_to_df(tracks):
    return pd.DataFrame([{
        "Track": t['name'],
        "Artist": ", ".join(a['name'] for a in t['artists']),
        "Preview": t['preview_url'],
        "Spotify Link": t['external_urls']['spotify']
    } for t in tracks])

In [34]:
def _fallback_to_playlist(sp, genre, limit=10):
    print(f"\nFalling back to public playlists for genre: '{genre}'")
    try:
        results = sp.search(q=genre, type='playlist', limit=5)
        if not results or not isinstance(results, dict):
            print(" Spotify search returned nothing useful.")
            return pd.DataFrame()
        playlists_section = results.get('playlists')
        if not playlists_section or not isinstance(playlists_section, dict):
            print(" 'playlists' section missing or invalid in result.")
            return pd.DataFrame()
        playlists = playlists_section.get('items', [])
        if not playlists or not isinstance(playlists, list):
            print(" No playlist items found.")
            return pd.DataFrame()
        for pl in playlists:
            pid = pl.get('id')
            if not pid:
                continue
            name = (pl.get('name') or '').strip() or 'Unnamed'
            print(f" Inspecting playlist: {name} (ID: {pid})")
            try:
                tracks_data = sp.playlist_tracks(pid)
            except Exception as e:
                print(f"  Could not fetch playlist {pid}: {e}")
                continue
            track_items = tracks_data.get('items', [])
            if not track_items or not isinstance(track_items, list):
                continue
            cleaned = [
                t['track'] for t in track_items
                if isinstance(t, dict)
                and isinstance(t.get('track'), dict)
                and t['track'].get('preview_url')
            ]
            if cleaned:
                print(f" Found {len(cleaned)} previewable tracks in '{name}'.")
                return pd.DataFrame([{
                    "Track": t["name"],
                    "Artist": ", ".join(a["name"] for a in t["artists"]),
                    "Preview": t["preview_url"],
                    "Spotify Link": t["external_urls"]["spotify"]
                } for t in cleaned[:limit]])
        print(" No usable tracks found in any fallback playlists.")
    except Exception as e:
        print(f" Total playlist fallback failure: {e}")
    return pd.DataFrame()

In [35]:
def fetch_spotify_tracks(sp, genre, limit=10):
    VALID_GENRE = genre_fallbacks.get(genre.lower(), genre.lower())
    if VALID_GENRE not in VALID_SEEDS:
        print(f" Genre '{VALID_GENRE}' invalid; using 'chill' instead.")
        VALID_GENRE = "chill"
    all_found = []
    try:
        print(f" Searching for genre:'{VALID_GENRE}' tracks")
        res = sp.search(q=f'genre:"{VALID_GENRE}"', type='track', limit=50)
        items = res.get('tracks', {}).get('items', []) or []
        all_found = [t for t in items if t.get('preview_url')]
    except Exception as e:
        print(f"  Genre search failed: {e}")
    if len(all_found) < limit:
        try:
            print(f" Expanding with keyword: {VALID_GENRE}")
            res2 = sp.search(q=VALID_GENRE, type='track', limit=50)
            items2 = res2.get('tracks', {}).get('items', []) or []
            extra = [t for t in items2 if t.get('preview_url') and t not in all_found]
            all_found.extend(extra)
        except Exception as e:
            print(f"  Keyword search failed: {e}")
    if len(all_found) >= limit:
        selected = random.sample(all_found, limit)
        return pd.DataFrame([{
            "Track": t["name"],
            "Artist": ", ".join(a["name"] for a in t["artists"]),
            "Preview": t["preview_url"],
            "Spotify Link": t["external_urls"]["spotify"]
        } for t in selected])
    print(" No previewable tracks found via standard search.")
    return pd.DataFrame()

In [36]:
def _ai_emotion_search(sp, emotion_list, limit=10):
    """
    Search Spotify by emotion keywords using emotion labels as queries.
    This fallback uses the AI-classified emotions directly as search terms.
    """
    print("\nAI-powered emotion search activated.")
    collected = []
    for emotion, score in emotion_list:
        try:
            print(f" • Searching for tracks related to: '{emotion}' ({score:.1%})")
            res = sp.search(q=emotion, type="track", limit=20)
            tracks = res.get("tracks", {}).get("items", []) or []
            previewables = [t for t in tracks if t.get("preview_url")]
            for t in previewables:
                if t not in collected:
                    collected.append(t)
            if len(collected) >= limit:
                break
        except Exception as e:
            print(f"   Failed query for '{emotion}': {e}")
    if not collected:
        print("No tracks found using AI emotion fallback.")
        return pd.DataFrame()
    sampled = random.sample(collected, min(limit, len(collected)))
    return format_tracks_to_df(sampled)

In [37]:
def mood_bot():
    user_input = input("Tell me how you're feeling: ")
    try:
        refined_emotions = classify_emotion_refined(user_input)
        print("\nTop Detected Emotions:")
        for label, score in refined_emotions:
            print(f"  {label}: {score:.2%}")
        mood, genre = map_emotion_to_genre(refined_emotions)
        print(f"\nMapped Mood: {mood}")
        print(f"Suggested Genre: {genre}")
        playlist_df = fetch_spotify_tracks(sp, genre)
        if not playlist_df.empty:
            print("\nHere's your personalized playlist:\n")
            display(playlist_df)
            return
        print("\nNo luck with direct track search. Trying public playlists...")
        playlist_df = _fallback_to_playlist(sp, genre)
        if not playlist_df.empty:
            print("\nHere's your playlist scraped from public collections:\n")
            display(playlist_df)
            return
        try:
            print("\nTrying Spotify's built-in recommendations...")
            rec = sp.recommendations(seed_genres=[genre], limit=50)
            rec_tracks = rec.get("tracks", []) or []
            rec_filtered = [t for t in rec_tracks if t.get("preview_url")]
            if rec_filtered:
                sampled = random.sample(rec_filtered, min(10, len(rec_filtered)))
                playlist_df = format_tracks_to_df(sampled)
                print("\nSpotify's algorithm came through after all:\n")
                display(playlist_df)
                return
        except Exception as e:
            print(f"Spotify recommendations failed: {e}")
        print("\nAll Spotify logic failed. Using AI to find tracks based on emotion keywords...")
        ai_df = _ai_emotion_search(sp, refined_emotions)
        if not ai_df.empty:
            print("\nHere's your AI-curated playlist:\n")
            display(ai_df)
            return
        print("\nAll systems failed. AI and Spotify have nothing left to give you. Good luck out there.")
    except Exception as e:
        print(f"\n Unexpected mood bot failure: {e}")

In [38]:
mood_bot()

Tell me how you're feeling:  I just got dumped and everything tastes like dust


Couldn't read cache at: .cache
Couldn't read cache at: .cache
Couldn't read cache at: .cache
Couldn't read cache at: .cache
Couldn't read cache at: .cache
Couldn't read cache at: .cache
Couldn't read cache at: .cache



Top Detected Emotions:
  disappointment: 39.27%
  disgust: 13.84%
  embarrassment: 9.62%

Mapped Mood: pensive
Suggested Genre: indie
 Searching for genre:'alternative' tracks
  Genre search failed: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions
 Expanding with keyword: alternative
  Keyword search failed: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions
 No previewable tracks found via standard search.

No luck with direct track search. Trying public playlists...

Falling back to public playlists for genre: 'indie'
 Total playlist fallback failure: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions

Trying Spotify's built-in recommendations...
Spotify recommendations failed: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions

All Spotify logic failed. Using AI to find tracks based 